In [ ]:
# (Run this / Run all) Import libraries
import os
from datetime import datetime
import zipfile

In [ ]:
# Enable functions to zip output files
def get_file_list(directory):
    file_list = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            if not file.endswith('.zip'):
                file_path = os.path.join(root, file)
                file_list.append(file_path)
    return file_list

def create_zip(file_list, zip_name, output_directory):
    zip_path = os.path.join(output_directory, zip_name)
    with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for file in file_list:
            arcname = os.path.relpath(file, output_directory)
            zipf.write(file, arcname)

In [ ]:
# (Run this) Zip all files and folders in /outputs except zip files
try:
    output_path = '/notebooks/stable-diffusion-webui/outputs'
    zip_name = f"outputs_{datetime.now().strftime('%Y%m%d%H%M%S')}.zip"

    file_list = get_file_list(output_path)
    create_zip(file_list, zip_name, output_path)

    print('Completed: All files and folders in "outputs" directory have been zipped, excluding existing zip files.')
except Exception as e:
    print(f'Error occurred: {str(e)}')

In [ ]:
### (Alternative for notebook) Uncomment and run to export all folders and files inside outputs
# (Alternative for terminal) Uncomment, then remove '%' and '!'
# %cd /notebooks/stable-diffusion-webui/outputs
# !zip -r $("outputs_" + date +"%Y%m%d%H%M%S").zip .